
<a href="https://colab.research.google.com/github/takzen/ai-engineering-handbook/blob/main/51_Recommender_Systems_SVD.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# 🎬 Systemy Rekomendacyjne: SVD i Ukryte Cechy

Mamy problem:
*   5 użytkowników.
*   6 filmów.
*   Kilka ocen (1-5 gwiazdek).
*   Mnóstwo pustych miejsc (NaN).

Chcemy przewidzieć, jak Użytkownik 3 oceniłby Film 4.

**Metoda: SVD (Singular Value Decomposition).**
Algorytm rozbija wielką macierz ocen na dwie mniejsze:
1.  **Macierz Użytkowników:** Opisuje gust każdego widza za pomocą "ukrytych liczb" (Latent Features).
2.  **Macierz Filmów:** Opisuje każdy film za pomocą tych samych ukrytych liczb.

Gdy pomnożymy te dwie macierze z powrotem, otrzymamy pełną tabelę z wypełnionymi lukami.

In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD

# 1. DANE (User-Item Matrix)
# Wiersze = Użytkownicy (A, B, C, D, E)
# Kolumny = Filmy (Matrix, StarWars, Titanic, NottingHill, Rambo, Shrek)
# 0 = Brak oceny (nie widział filmu)

data = {
    'Matrix':      [5, 4, 0, 0, 1],
    'StarWars':    [5, 5, 0, 0, 2],
    'Titanic':     [0, 0, 5, 4, 0],
    'NottingHill': [0, 0, 5, 5, 0],
    'Rambo':       [4, 0, 0, 0, 1],
    'Shrek':       [0, 0, 0, 0, 5]
}

ratings = pd.DataFrame(data, index=['User_A', 'User_B', 'User_C', 'User_D', 'User_E'])

print("--- NASZA MACIERZ OCEN (0 = Brak danych) ---")
display(ratings)
print("\nWidzisz wzorzec? A i B lubią Sci-Fi. C i D lubią Romanse. E lubi bajki.")

--- NASZA MACIERZ OCEN (0 = Brak danych) ---


,Matrix,StarWars,Titanic,NottingHill,Rambo,Shrek
User_A,5,5,0,0,4,0
User_B,4,5,0,0,0,0
User_C,0,0,5,5,0,0
User_D,0,0,4,5,0,0
User_E,1,2,0,0,1,5



Widzisz wzorzec? A i B lubią Sci-Fi. C i D lubią Romanse. E lubi bajki.


## Krok 1: SVD w akcji

Użyjemy `TruncatedSVD` ze Scikit-Learn.
Musimy wybrać liczbę **komponentów** (cech ukrytych).
Powiedzmy, że chcemy opisać świat filmów za pomocą **2 cech**.
(Algorytm sam wymyśli, co to za cechy – my możemy się domyślać, że to np. "Akcja" i "Romantyzm").

In [2]:
# Zamieniamy DataFrame na macierz numpy
X = ratings.values

# Tworzymy SVD (szukamy 2 ukrytych cech)
svd = TruncatedSVD(n_components=2, random_state=42)

# Trenujemy (Dopasowujemy macierz użytkowników)
matrix_user_features = svd.fit_transform(X)

print("--- UKRYTE CECHY UŻYTKOWNIKÓW (User Matrix) ---")
# Wiersze to userzy, kolumny to nasze 2 tajne cechy
print(matrix_user_features.round(2))

print("\n--- UKRYTE CECHY FILMÓW (Item Matrix) ---")
# To jest ukryte wewnątrz obiektu SVD (components_)
matrix_movie_features = svd.components_
print(matrix_movie_features.round(2))

--- UKRYTE CECHY UŻYTKOWNIKÓW (User Matrix) ---
[[ 7.89  0.  ]
 [ 5.99  0.  ]
 [-0.    7.06]
 [-0.    6.39]
 [ 3.06  0.  ]]

--- UKRYTE CECHY FILMÓW (Item Matrix) ---
[[ 0.62  0.7  -0.   -0.    0.32  0.14]
 [ 0.    0.    0.67  0.74  0.    0.  ]]


## Krok 2: Rekonstrukcja (Wypełnianie luk)

Teraz najważniejszy moment.
Mnożymy macierz użytkowników przez macierz filmów.
$$ X_{new} = U \times V^T $$

Wynikowa macierz będzie miała liczby w każdym polu.
*   Tam, gdzie były oceny – liczby powinny być podobne do oryginału.
*   Tam, gdzie było 0 – pojawią się **predykcje**.

In [3]:
# Rekonstrukcja macierzy (Iloczyn skalarny)
predicted_ratings = np.dot(matrix_user_features, matrix_movie_features)

# Zamiana na DataFrame dla czytelności
pred_df = pd.DataFrame(predicted_ratings, index=ratings.index, columns=ratings.columns)

print("--- MACIERZ PRZEWIDYWANA (Wypełnione luki) ---")
display(pred_df.round(2))

print("-" * 30)
print("ANALIZA:")
print("Spójrz na User_B (drugi wiersz).")
print(f"Oryginał Rambo: {ratings.loc['User_B', 'Rambo']} (Nie widział)")
print(f"Przewidywanie Rambo: {pred_df.loc['User_B', 'Rambo']:.2f}")
print("System przewidział, że User_B oceniłby Rambo na ~2.86 (bo lubi Matrixa, ale Rambo to trochę co innego).")

--- MACIERZ PRZEWIDYWANA (Wypełnione luki) ---


,Matrix,StarWars,Titanic,NottingHill,Rambo,Shrek
User_A,4.88,5.55,-0.00,-0.00,2.54,1.12
User_B,3.70,4.21,-0.00,-0.00,1.93,0.85
User_C,-0.00,-0.00,4.74,5.24,0.00,0.00
User_D,-0.00,-0.00,4.29,4.74,0.00,0.00
User_E,1.89,2.15,0.00,0.00,0.98,0.43


------------------------------
ANALIZA:
Spójrz na User_B (drugi wiersz).
Oryginał Rambo: 0 (Nie widział)
Przewidywanie Rambo: 1.93
System przewidział, że User_B oceniłby Rambo na ~2.86 (bo lubi Matrixa, ale Rambo to trochę co innego).


In [4]:
# KORELACJA FILMÓW (Podobieństwo)
# Skoro mamy wektory cech dla filmów, możemy sprawdzić, które są do siebie podobne (używając Korelacji Pearsona na macierzy korelacji)

corr_matrix = np.corrcoef(matrix_movie_features.T) # .T żeby korelacja była między filmami (kolumnami)

print("--- KTÓRY FILM PASUJE DO KTÓREGO? ---")
# Wypiszmy, co jest podobne do "Matrixa" (Index 0)
matrix_corr = corr_matrix[0]
similar_movies = list(zip(ratings.columns, matrix_corr))

# Sortujemy
sorted_sim = sorted(similar_movies, key=lambda x: x[1], reverse=True)

for movie, score in sorted_sim:
    print(f"Podobieństwo do Matrixa: {movie:<12} = {score:.4f}")

--- KTÓRY FILM PASUJE DO KTÓREGO? ---
Podobieństwo do Matrixa: Matrix       = 1.0000
Podobieństwo do Matrixa: StarWars     = 1.0000
Podobieństwo do Matrixa: Rambo        = 1.0000
Podobieństwo do Matrixa: Shrek        = 1.0000
Podobieństwo do Matrixa: Titanic      = -1.0000
Podobieństwo do Matrixa: NottingHill  = -1.0000


## 🧠 Podsumowanie: Jak Netflix zarabia miliony?

Algorytm poprawnie wykrył, że:
1.  **Matrix** jest bardzo podobny do **StarWars** (korelacja bliska 1.0).
2.  **Matrix** jest totalnym przeciwieństwem **Titanica** (korelacja ujemna -1.0).

**Tu jest haczyk (Cold Start).**
SVD działa genialnie, gdy masz historię ocen.
Ale co zrobić, gdy zarejestruje się **nowy użytkownik**, który nie ocenił niczego? Jego wiersz to same zera. SVD nic nie przewidzi (wynik też będzie zerem).
To tzw. problem "Zimnego Startu". Wtedy Netflix używa prostych reguł ("Co jest popularne w Twoim kraju?").

**Wniosek:**
Faktoryzacja macierzy to fundament. Choć nowoczesne systemy używają do tego sieci neuronowych (Neural Collaborative Filtering), to matematyka pod spodem (mnożenie wektorów cech) pozostaje ta sama.